In [5]:
# Parser unificato e robusto per run_PVD_SiC + analyze_PVD_SiC → NeXus
import os, subprocess, datetime, traceback
import numpy as np
import h5py
from pymulskips import setuprun, process
from pymulskips.analyze import analyze_growth_rate
from analyze_PVD_SiC import count_vacancies

# Funzione helper per dataset con unità
def set_ds(group, name, value, unit=None):
    ds = group.create_dataset(name, data=value)
    if unit:
        ds.attrs['units'] = unit
    return ds

# === INPUT DINAMICO ===
def ask_value(prompt, default, cast):
    val = input(f"{prompt} [default: {default}]: ")
    return cast(val) if val.strip() else default

sample_id = ask_value("ID del campione", '153', str)
lenx = ask_value("Lunghezza X", 120, int)
leny = ask_value("Lunghezza Y", 120, int)
lenz = ask_value("Lunghezza Z", 480, int)
randseed = ask_value("Seed random", 9117116, int)
newdT = ask_value("Delta T aggiuntivo (newdT)", 0, float)
dT = ask_value("Delta T da aggiungere a Tsource", 100, float)
method = ask_value("Metodo di analisi (polyfit o average)", 'polyfit', str)
bin_size = ask_value("Bin size (cm)", 0.001, float)
surface_roughness = ask_value("Filtro rugosità superficie", 0.5, float)
Nexclude = ask_value("Nexclude (default 3)", 3, int)

# === COSTANTI E PARAMETRI ===
execpath = '/Users/filipporuberto/miniforge3/envs/mulskips_env/MulSKIPS/mulskips-source'
DATA_SAMPLES = {
    '153': {
        'Power': 6.10,
        'T_seed_middle': 2072.24,
        'T_seed_edge': 0,
        'T_source_middle': 2080.20,
        'T_source_edge': 2080.20,
        'Exp-Growth-Rate': 287
    }
}
data = dict(DATA_SAMPLES[sample_id])
tseed = data['T_seed_middle'] + 273.15 + newdT
tsource_0 = data['T_source_middle'] + 273.15 + newdT
tsource = tsource_0 + dT
gr = data['Exp-Growth-Rate'] * (dT / 25 if dT != 0 else 1)
KMC_lattice_constant = 0.436 / 12
alat = 5.43
target_thickness = 0.9 * lenz * KMC_lattice_constant * 1e-3
tottime = 3600 * target_thickness / gr
Nout = 90
runpath = f"parser/data-{dT}-{newdT}-{randseed}"
os.makedirs(runpath, exist_ok=True)

# === RUN MULSKIPS ===
setuprun.setup_mulskips_src(execpath, lenx, leny, lenz)
pvdclass = process.PVD(substrate='SiC-3C', precursors=['Si', 'Si2C', 'SiC2'], calibration_type='avrov', Tsource=tsource, Tseed_center=tseed)
setuprun.RunType = 'R'
setuprun.IDUM = randseed
setuprun.setup_only = False
setuprun.TotTime = tottime
setuprun.OutTime = tottime / Nout
setuprun.OutMolMol = 1
setuprun.Seed_box = [48, 0, 0]
setuprun.run_mulskips(execpath, runpath, 'F', pvdclass, PtransZig=0.93, ExitStrategy='Time', SaveCoo=False)

# === ANALISI ===
surf_file = os.path.join(runpath, "surf_height.txt")
xyz_file = os.path.join(runpath, f"{randseed}_v.xyz")
results_file = os.path.join(runpath, "results.txt")
logfile = os.path.join(runpath, "runlog.txt")

try:
    growth_rate, avg_surf_height = analyze_growth_rate(runpath, method=method, return_surf_height=True,
                                                       surface_roughness=surface_roughness, bin_size=bin_size,
                                                       Nexclude=Nexclude)
except Exception as e:
    print(f"⚠️ Growth rate fallita: {e}")
    growth_rate, avg_surf_height = np.nan, np.nan

vac_data = count_vacancies(runpath)

try:
    subprocess.run(f"cat {runpath}/start.dat > {logfile}", shell=True, check=True)
except Exception as e:
    print(f"⚠️ Log fallito: {e}")

def read_file(path, fallback="null"):
    try:
        with open(path, 'r') as f:
            return f.read()
    except:
        print(f"⚠️ File mancante: {path}")
        return fallback

xyz_txt = read_file(xyz_file)
results_txt = read_file(results_file)
log_txt = read_file(logfile)

# === SCRITTURA NEXUS ===
nexus_file = f"SiC_sample_{sample_id}.nxs"
try:
    with h5py.File(nexus_file, 'w') as f:
        f.attrs['NX_class'] = 'NXroot'
        f.create_dataset('default', data=b'NXmicrostructure_imm_results')

        def grp(name):
            g = f.create_group(name)
            g.attrs['NX_class'] = name
            return g

        software = grp('NXsoftware')
        set_ds(software, 'name', b'MulSKIPS')
        set_ds(software, 'version', b'1.0')
        set_ds(software, 'description', b'Multiscale KMC for PVD SiC')

        user = grp('NXuser')
        set_ds(user, 'name', b'Filippo Ruberto')
        set_ds(user, 'affiliation', b'CNR-IMM@CT')
        set_ds(user, 'email', b'filippo.ruberto@cnr.it')
        set_ds(user, 'ORCID', b'0000-0002-1234-5678')

        project = grp('NXproject')
        set_ds(project, 'name', b'MulSKIPS@CNR-IMM')
        set_ds(project, 'description', b'Multiscale modeling of PVD growth of SiC')
        set_ds(project, 'grant_number', b'FAIRmat-NFDI')

        prov = grp('NXprovenance')
        set_ds(prov, 'source', b'https://github.com/mulskips/mulskips-source')
        set_ds(prov, 'modification_date', str(datetime.datetime.now()))

        conf = grp('NXmicrostructure_imm_config')
        set_ds(conf, 'power', data['Power'], 'W')
        set_ds(conf, 'temperature_seed', tseed, 'K')
        set_ds(conf, 'temperature_source', tsource, 'K')
        set_ds(conf, 'initial_temperature_source', tsource_0, 'K')
        set_ds(conf, 'temperature_difference', dT, 'K')
        set_ds(conf, 'additional_temperature_shift', newdT, 'K')
        set_ds(conf, 'geometry/lenx', lenx, 'lattice units')
        set_ds(conf, 'geometry/leny', leny, 'lattice units')
        set_ds(conf, 'geometry/lenz', lenz, 'lattice units')
        set_ds(conf, 'KMC_lattice_constant', KMC_lattice_constant, 'nm')
        set_ds(conf, 'alat_angstrom', alat, 'Angstrom')
        set_ds(conf, 'simulation_type', b'PVD')
        set_ds(conf, 'substrate_material', b'SiC-3C')
        conf.create_dataset('precursors', data=np.array([b'Si', b'Si2C', b'SiC2']))
        set_ds(conf, 'experiment_identifier', f'PVD-SiC-Run-{sample_id}'.encode())
        set_ds(conf, 'deposition_time', tottime, 's')
        set_ds(conf, 'output_steps', Nout)
        set_ds(conf, 'target_thickness', target_thickness, 'mm')
        set_ds(conf, 'random_seed', randseed)
        set_ds(conf, 'timestamp', str(datetime.datetime.now()))
        set_ds(conf, 'run_directory', runpath.encode())

        # Parametri avanzati
        set_ds(conf, 'simulation_parameters/PtransZig', 0.93)
        set_ds(conf, 'simulation_parameters/ExitStrategy', b'Time')
        set_ds(conf, 'simulation_parameters/SaveCoo', b'False')
        conf.create_dataset('simulation_parameters/Seed_box', data=[48, 0, 0])
        set_ds(conf, 'simulation_parameters/setup_only', b'False')
        set_ds(conf, 'analysis/bin_size', bin_size, 'cm')
        set_ds(conf, 'analysis/surface_roughness', surface_roughness)
        set_ds(conf, 'analysis/Nexclude', Nexclude)

        res = grp('NXmicrostructure_imm_results')
        set_ds(res, 'growth_rate', 0.0 if np.isnan(growth_rate) else growth_rate, 'micron/hour')
        set_ds(res, 'avg_surface_height', 0.0 if np.isnan(avg_surf_height) else avg_surf_height, 'micron')
        set_ds(res, 'growth_rate_method', np.string_(method))
        set_ds(res, 'experimental_growth_rate', data['Exp-Growth-Rate'], 'micron/hour')
        set_ds(res, 'alat_used', alat, 'Angstrom')
        set_ds(res, 'analysis_time', str(datetime.datetime.now()))
        set_ds(res, 'total_time', tottime, 's')
        set_ds(res, 'num_outputs', Nout)
        set_ds(res, 'target_thickness', target_thickness, 'mm')
        set_ds(res, 'vacancies_total', vac_data.get('total', -1))

        for k, v in vac_data.items():
            set_ds(res, f'vacancy_{k}', v)

        set_ds(res, 'xyz_file', np.string_(xyz_txt))
        set_ds(res, 'results_txt', np.string_(results_txt))

        try:
            lines = results_txt.strip().splitlines()
            hdrs = lines[0].split()
            data_mat = np.array([list(map(float, l.split())) for l in lines[1:] if l.strip()])
            for i, h in enumerate(hdrs):
                set_ds(res, f'results_table/{h}', data_mat[:, i])
        except Exception as e:
            print(f"⚠️ Tabella results.txt fallita: {e}")

        try:
            txt = read_file(surf_file)
            tab = np.loadtxt(surf_file)
            if tab.ndim == 1:
                tab = tab.reshape(-1, 2)
            set_ds(res, 'surf_height_txt', np.string_(txt))
            set_ds(res, 'surf_height/time', tab[:, 0], 's')
            set_ds(res, 'surf_height/height', tab[:, 1], 'micron')
        except Exception as e:
            print(f"⚠️ surf_height.txt fallita: {e}")
            set_ds(res, 'surf_height_txt', np.string_("null"))

        log = grp('simulation_log')
        set_ds(log, 'description', b'MulSKIPS simulation log')
        set_ds(log, 'data', np.string_(log_txt))

    print(f"✅ File NeXus scritto con successo: {nexus_file}")
except Exception as e:
    print(f"❌ Errore nella scrittura del NeXus: {e}")
    traceback.print_exc()


ID del campione [default: 153]:  
Lunghezza X [default: 120]:  120
Lunghezza Y [default: 120]:  120
Lunghezza Z [default: 480]:  2400
Seed random [default: 9117116]:  
Delta T aggiuntivo (newdT) [default: 0]:  
Delta T da aggiungere a Tsource [default: 100]:  0
Metodo di analisi (polyfit o average) [default: polyfit]:  
Bin size (cm) [default: 0.001]:  
Filtro rugosità superficie [default: 0.5]:  
Nexclude (default 3) [default: 3]:  


Compiling MulSKIPS with box size: 120 x 120 x 2400
Directory changed to: /Users/filipporuberto/miniforge3/envs/mulskips_env/MulSKIPS/mulskips-source

********************************** Starting MulSKIPS compilation **********************************
MulSKIPS:	rm ./modules/defdertype.o ./modules/defsystem.o ./modules/definitions.o ./initial/SetAPB.o ./initial/SetInvPC.o ./initial/SetInvPwAPB.o ./initial/SetNCSp.o ./initial/SetNCSpSi.o ./initial/SetAPBSym.o ./initial/SetInvP.o ./initial/SetInvPwOnlyCorSi.o ./initial/SetSiC3C.o ./initial/SetSiCSF.o ./initial/SetSiGex.o ./initial/SetSiC3Cww.o ./initial/SetSiC3Cwwaperture.o ./initial/SetSi.o ./initial/SetSiFINFET.o ./initial/SetCAD.o ./initial/SetT.o ./initial/SetShape.o ./initial/SetAPBSymZimb1.o ./initial/SetInvPOff.o ./initial/SetInvPwOnlySiorC.o ./initial/SetTrenchSi.o ./initial/SetTrench.o ./initial/SetAPBSymZimb.o ./initial/SetInvPSi.o ./initial/SetNC.o ./inpout/FileClose.o ./inpout/GetOutputFileName.o ./inpout/WriteMolMolXYZFile.o ./

ld: warning: ignoring duplicate libraries: '-lgfortran', '-lquadmath'


********************************** Finished MulSKIPS compilation **********************************

Directory changed to: /Users/filipporuberto/miniforge3/envs/mulskips_env/MulSKIPS
*** Initializing PVD process for SiC-3C substrate with precursors ['Si', 'Si2C', 'SiC2']
precursor_masses 	--> Precursor Masses [kg/mol]: {'Si': 0.028085000000000002, 'Si2C': 0.068181, 'SiC2': 0.052107}
mass 			--> Substrate Mass [Kg/mol]: 0.040096
rho 			--> Substrate Density [Kg/m^3]: 3210
rho_surf 		--> Substrate Surface density [Kg/m^2]: 1.400038787156981e-06
calibration_type 	--> Calibration of fluxes taken from:  avrov
Tsource 	--> Temperature at source:  2353.35
Tseed_center 	--> Temperature at seed center: 2345.39
KMC_sf 			--> Substrate KMC super-lattice constant [angstrom]: 0.36333333333333334
listcry 		--> Crystalline species in the substrate: ['Si', 'C']
listcryZ 		--> Atomic number of Crystalline species in the substrate: [14, 6]
calibration_params 	--> Calibration parameters:
     fact_D_Si 	